### Process 150 demonstrations from dat file to JSON

In [4]:
import os
import sys
import pickle

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [32]:
name_to_folder = {
    'Around' : 'SlideAroundDiscrete',
    'Closer' : 'SlideToward',
    'Next' : 'SlideNext',
    'Past' : 'SlidePast',
    'Away' : 'SlideAway',
}

json_structure = {}


for action in name_to_folder:
    json_structure[action] = []
    
    prefix = os.path.join( "..", "experiments", 
                          "human_evaluation_2d", 
                          name_to_folder[action])

    for index in range(30):
        one_demo = {}
        stored_config_file = os.path.join(prefix, str(index) + ".dat")
        with open(stored_config_file, 'rb') as fh:
            # need this encoding 
            stored_config = pickle.load(fh, encoding='latin-1')
    
        one_demo ['start_config']  = [list(cube.transform.position.flatten()) for cube in stored_config['start_config'] ]
        
        one_demo ['action_storage'] = []
        
        for _, _, next_transform, _, _, success, _, _ in stored_config['action_storage']:
            if success:
                t = list(next_transform.position.flatten())
                if len(one_demo ['action_storage']) == 0 or \
                        (len(one_demo ['action_storage']) > 0 and one_demo ['action_storage'][-1] != t):
                    one_demo ['action_storage'].append( t )
        
        json_structure[action].append(one_demo)

In [27]:
import json

In [33]:
with open('all_demos.json', 'w') as fh:
    json.dump(json_structure, fh)

In [35]:
import numpy as np
np.random.permutation(150)

array([121,  82,  91,  46,  21,   2,  94,  13,  78,  74,  64,  93,  76,
        49,  66, 113, 126, 132,   3, 100,  68,  20, 115,  99, 124, 102,
        83,  34, 109,  10,  25,  58,  96,  90, 144,  26,  32,  22,  24,
       131,  88,  97,  38,  31,  87, 134,  54,  17, 122,  40,  50,  61,
        42,  36, 123,  51,  56,  44,   4,   5,  15, 125, 106,  95, 143,
       147,  79, 139, 130,  60, 112, 116, 133,  35, 137,   8,  80,  84,
       105, 118,  62,  39,  85,  63, 117,  72,  14,  92,  71,  43, 140,
        67,  12,  69,  11, 110, 108, 145,  57, 114, 138, 127, 128,  65,
        52, 141,  81,  86,  53, 136,  75,  55,  47, 129, 120,  41,   0,
        73,   9, 104,  89,  28,   1,  16, 148,  29,  77, 149,  18, 111,
        59,  37, 101,  33, 119,  19,  23,  45, 107,   6,  30,  70, 142,
         7,  27, 135, 146,  98, 103,  48])